In [1]:
from pathlib import Path
import os
from dotenv import load_dotenv
from tqdm import tqdm
tqdm.pandas()
%load_ext autoreload
%autoreload 2

In [2]:
# change dir to ml repo at ml/ root (if you started in notebook/)
os.chdir('../')
print(Path.cwd())
load_dotenv()

/home/opadmin/llama2d_opentable/lm_act_eval


True

In [3]:
from datasets import Dataset
import pandas as pd
from lm_act_eval.evaluation_harness.helper_functions.multion import (
  action_prefix,
  clean_extracted_text,
  extract_thought,
  extract_action,
  extract_explanation,
  ParseChatCompletion,
  extract_first
)
from lm_act_eval.evaluation_harness.utils.url import is_screenshot_url_accessible

/home/opadmin/anaconda3/envs/eval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Google Cloud not set up, skipping import of providers.gemini_utils.generate_from_gemini_completion
Google Cloud not set up, skipping import of vertexai.preview.generative_models.Image and llms.generate_from_gemini_completion


/home/opadmin/anaconda3/envs/eval/lib/python3.11/site-packages/deepeval/__init__.py:41: UserWarning: You are using deepeval version 0.20.90, however version 0.21.21 is available. You should consider upgrading via the "pip install --upgrade deepeval" command.
  warnings.warn(


In [11]:
from typing import Callable
from tqdm import tqdm

In [12]:
# subdependent 
from lm_act_eval.evaluation_harness.evaluators.sft.dataframe import DataFrameEvaluator
from lm_act_eval.evaluation_harness.helper_functions import multion

In [23]:
from lm_act_eval._init_conf import init_conf
hydra_cfg = init_conf.main(
  standalone_mode=False, args=['trajectory_eval'])
cfg = hydra_cfg.eval

Composed Configurations:
project: opentable
eval:
  sft:
    trajectory:
      data:
        path: lm_act_eval/.cache/five-star-trajectories/csv/data+gptv-eligible.csv
        columns:
          'y': ground_truth
          y_': GPTV_generations
        extract_fs:
          QUERY:
            QUERY: null
          screenshot:
            screenshot: null
          GOAL:
            chat_completion_messages: parse_completion.parse_content
        logging:
          wandb:
            project: opentable
            result: lm_act_eval-run
          braintrust:
            project: multion_opentable
      comparator:
        gpt-v:
          model: gpt-4-vision-preview
          max_token: 300
          img_fidelity: high
      metrics:
        gpt-v:
          inputs:
          - GOAL
          - QUERY
          - screenshot
          args: null
        llm_relevancy:
        - explanation



In [24]:
df = pd.read_csv(cfg.sft.trajectory.data.path)
print(
  df.chat_completion_messages.iloc[0][:100],
  df.columns
)

[{'role': 'system', 'content': '**Goal**: Let\'s play a game - You are an expert agent named MULTI·O Index(['Unnamed: 0', 'session_id', 'idx_in_session', '_id', 'event_id',
       'parent_id', 'event_type', 'event_name', 'config', 'children_ids',
       'inputs', 'outputs', 'error', 'start_time', 'end_time', 'metadata',
       'feedback', 'metrics', 'user_properties', 'source', 'duration',
       'project_id', 'tenant', 'QUERY', 'URL', 'DOM', 'RULES', 'USER_CONTEXT',
       'PREV_ACTIONS', 'chat_history', 'CURRENT_TIME', 'planPrompt',
       'previousActionsRepetitionPrompt', 'chat_completion_messages',
       'ground_truth', 'screenshot', 'prompt', 'internal_metadata',
       'chat_completion_messages_content', 'GPTV_generations'],
      dtype='object')


In [25]:
df.QUERY.iloc[0]

'What was the actual enrollment count of the clinical trial on H. pylori in acne vulgaris patients from Jan-May 2018 as listed on the NIH website?'

In [6]:
from lm_act_eval.evaluation_harness.openai.vision.evaluator import GPTVScorer

In [28]:
cfg.sft.trajectory.metrics

{'gpt-v': {'inputs': ['GOAL', 'QUERY', 'screenshot'], 'args': None}, 'llm_relevancy': ['explanation']}

In [37]:
from lm_act_eval.evaluation_harness.evaluators.sft.trajectory import TableTrajectoryEvaluator

df_evaluator = TableTrajectoryEvaluator(cfg.sft.trajectory)

TypeError: argument of type 'NoneType' is not iterable

In [14]:
df_evaluator.df.head(2)

,session_id,idx_in_session,_id,event_id,parent_id,event_type,event_name,config,children_ids,inputs,...,CURRENT_TIME,planPrompt,previousActionsRepetitionPrompt,chat_completion_messages,ground_truth,screenshot,prompt,internal_metadata,chat_completion_messages_content,GPTV_generations
169,36c2f0ba-e215-4081-96b4-ecb04c10a517,0,65ed4fc8166e07fe3c29c476,6c8932f3-c31d-4683-aeb0-52a5bbe5a978,36c2f0ba-e215-4081-96b4-ecb04c10a517,model,Model Completion,"{'model': 'gpt-4-1106-preview', 'provider': 'u...",[],{'QUERY': 'What was the actual enrollment coun...,...,"Sunday, 10 March 2024 at 17:14:14 GMT+11","IMPORTANT: If the objective is complex, compri...",**PREVIOUS ACTIONS Repetition**:\n- Review the...,"[{'role': 'system', 'content': '**Goal**: Let\...",PLAN:\n1. Use the Google search function to fi...,https://multion-client-screenshots.s3.us-east-...,NaN,NaN,**Goal**: Let's play a game - You are an exper...,PLAN: The current user objective is to find th...
170,36c2f0ba-e215-4081-96b4-ecb04c10a517,1,65ed4fd1b2b5b0c1673c47d7,172ff25e-efdb-4002-b43a-0180f3f0eb19,36c2f0ba-e215-4081-96b4-ecb04c10a517,model,Model Completion,"{'model': 'gpt-4-1106-preview', 'provider': 'u...",[],{'QUERY': 'What was the actual enrollment coun...,...,"Sunday, 10 March 2024 at 17:14:35 GMT+11","IMPORTANT: If the objective is complex, compri...",**PREVIOUS ACTIONS Repetition**:\n- Review the...,"[{'role': 'system', 'content': '**Goal**: Let\...",COMMANDS:\nCLICK 26\nEXPLANATION: I am clickin...,https://multion-client-screenshots.s3.us-east-...,NaN,NaN,**Goal**: Let's play a game - You are an exper...,COMMANDS:\nCLICK 26\nEXPLANATION: I am clickin...


In [ ]:
gptv_scorer = GPTVScorer(hydra_cfg.sft.trajectory.metrics['gpt-v'])
hydra_cfg.sft.trajectory.metrics['gpt-v']

{'inputs': ['GOAL', 'QUERY', 'screenshots'], 'args': None}

In [ ]:
# 
df_evaluator.df.iloc[:2]

,QUERY,screenshot,GOAL
169,What was the actual enrollment count of the cl...,https://multion-client-screenshots.s3.us-east-...,**Goal**: Let\'s play a game - You are an expe...
170,What was the actual enrollment count of the cl...,https://multion-client-screenshots.s3.us-east-...,**Goal**: Let\'s play a game - You are an expe...


In [ ]:
gptv_scorer(df_evaluator.df.iloc[:2])

Evaluating with GPT-V: 100%|██████████| 2/2 [00:46<00:00, 23.10s/it]


,Score,Explanation
169,70,The screenshot indicates that the user has suc...
170,90,The user is very close to completing their tas...


## Save/Load Data

In [ ]:
# traj_df_new = traj_df.merge(eligible_traj_df, how='left')
# traj_df_new.to_csv(dataset_path/'csv/data+gptv_eval.csv')

In [ ]:
table_name = 'data+gptv_eval-eligible'
artifact_name = 'gptv_opentable-trajectory_eval'
eligible_traj_df.to_csv(dataset_path/f'csv/{table_name}.csv')

In [ ]:
eligible_traj_df = pd.read_csv(
  dataset_path/f'csv/{table_name}.csv', index_col=0)
eligible_traj_df.index.name = None
eligible_traj_df.head(2)

,session_id,idx_in_session,_id,event_id,parent_id,event_type,event_name,config,children_ids,inputs,...,planPrompt,previousActionsRepetitionPrompt,chat_completion_messages,ground_truth,screenshot,prompt,internal_metadata,GPTV response,chat_completion_messages_content,GPTV_evaluation
169,36c2f0ba-e215-4081-96b4-ecb04c10a517,0,65ed4fc8166e07fe3c29c476,6c8932f3-c31d-4683-aeb0-52a5bbe5a978,36c2f0ba-e215-4081-96b4-ecb04c10a517,model,Model Completion,"{'model': 'gpt-4-1106-preview', 'provider': 'u...",[],{'QUERY': 'What was the actual enrollment coun...,...,"IMPORTANT: If the objective is complex, compri...",**PREVIOUS ACTIONS Repetition**:\n- Review the...,"[{'role': 'system', 'content': '**Goal**: Let\...",PLAN:\n1. Use the Google search function to fi...,https://multion-client-screenshots.s3.us-east-...,NaN,NaN,It appears that you are looking for informatio...,**Goal**: Let's play a game - You are an exper...,SCORE: 60\nEXPLANATION:\nThe user appears to b...
170,36c2f0ba-e215-4081-96b4-ecb04c10a517,1,65ed4fd1b2b5b0c1673c47d7,172ff25e-efdb-4002-b43a-0180f3f0eb19,36c2f0ba-e215-4081-96b4-ecb04c10a517,model,Model Completion,"{'model': 'gpt-4-1106-preview', 'provider': 'u...",[],{'QUERY': 'What was the actual enrollment coun...,...,"IMPORTANT: If the objective is complex, compri...",**PREVIOUS ACTIONS Repetition**:\n- Review the...,"[{'role': 'system', 'content': '**Goal**: Let\...",COMMANDS:\nCLICK 26\nEXPLANATION: I am clickin...,https://multion-client-screenshots.s3.us-east-...,NaN,NaN,The image shows a webpage from ClinicalTrials....,**Goal**: Let's play a game - You are an exper...,SCORE: 95\nEXPLANATION:\nThe user is very clos...


Logging

In [ ]:
# log data
import wandb
from dotenv import load_dotenv
load_dotenv()
wandb.login(relogin=True)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /home/opadmin/.netrc


True

In [ ]:
opentable_artifact = wandb.Artifact("opentable_trajectories-gptv_eval", type="dataset")
# opentable_table = wandb.Table(dataframe=traj_df_new)
opentable_table_eligible = wandb.Table(dataframe=eligible_traj_df)
#
# opentable_artifact.add(opentable_table, "opentable")
opentable_artifact.add(
  opentable_table_eligible, artifact_name)
# opentable_artifact.add_file(str(dataset_path/'csv/data+gptv.csv'))
opentable_artifact.add_file(str(dataset_path/f'csv/{table_name}.csv'))


In [ ]:
# Start a W&B run to log data
run = wandb.init(project="trajectory_eval", reinit=True)
# Log the table to visualize with a run...
run.log({"opentable_gptv": opentable_table_eligible})

# and Log as an Artifact to increase the available row limit!
run.log_artifact(opentable_artifact)

wandb: Currently logged in as: mmym (multion-agi). Use `wandb login --relogin` to force relogin


wandb: WARNING Serializing object of type str that is 531752 bytes
wandb: WARNING Serializing object of type str that is 517908 bytes
wandb: WARNING Serializing object of type str that is 541284 bytes
wandb: WARNING Serializing object of type str that is 525810 bytes
wandb: WARNING Serializing object of type str that is 116864 bytes
wandb: WARNING Serializing object of type str that is 121274 bytes
wandb: WARNING Serializing object of type str that is 120688 bytes
wandb: WARNING Serializing object of type str that is 124364 bytes
wandb: WARNING Serializing object of type str that is 127354 bytes
wandb: WARNING Serializing object of type str that is 129308 bytes
wandb: WARNING Serializing object of type str that is 129582 bytes
wandb: WARNING Serializing object of type str that is 135310 bytes
wandb: WARNING Serializing object of type str that is 137856 bytes
wandb: WARNING Serializing object of type str that is 137644 bytes
wandb: WARNING Serializing object of type str that is 136800 b

<Artifact opentable_trajectories-gptv_eval>